In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scibert/scibert/config.json
/kaggle/input/scibert/scibert/model.safetensors
/kaggle/input/kriti-2024/sample_submission.csv
/kaggle/input/kriti-2024/train.csv
/kaggle/input/kriti-2024/test.csv
/kaggle/input/scibert-final1/scibert_final/config.json
/kaggle/input/scibert-final1/scibert_final/model.safetensors


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from imblearn.ensemble import EasyEnsembleClassifier
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [10]:
df=pd.read_csv('/kaggle/input/kriti-2024/train.csv')
X = df[['Title', 'Abstract']]
y = df['Categories']
mlb = MultiLabelBinarizer()
y_binary = mlb.fit_transform(y.apply(eval))

In [11]:
test_df=pd.read_csv('/kaggle/input/kriti-2024/test.csv')

In [12]:
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased', do_lower_case=True)
scibert_model = BertForSequenceClassification.from_pretrained('/kaggle/input/scibert-final1/scibert_final')

In [13]:
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
        
    return text

In [14]:
def tokenize_text(text):
    return tokenizer.encode_plus(text, add_special_tokens=True, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

In [15]:
from tqdm import tqdm
import torch
scibert_model.to('cuda')
# Preprocess the test data
test_df['Processed_Text'] = [preprocess_text(title + ' ' + abstract) for title, abstract in zip(test_df['Title'], test_df['Abstract'])]

# Tokenize and preprocess the text data for test
X_test_tokens1 = [tokenize_text(text) for text in test_df['Processed_Text']]
X_test_inputs1 = torch.cat([x['input_ids'] for x in X_test_tokens1], dim=0)
X_test_attention_masks1 = torch.cat([x['attention_mask'] for x in X_test_tokens1], dim=0)

# Move test data to GPU
X_test_inputs1 = X_test_inputs1.to('cuda')
X_test_attention_masks1 = X_test_attention_masks1.to('cuda')

# Set batch size
batch_size = 9

# Split the test data into batches
X_test_inputs_batches1 = torch.split(X_test_inputs1, batch_size)
X_test_attention_masks_batches1= torch.split(X_test_attention_masks1, batch_size)

# Make predictions on the test data
y_pred_list = []

scibert_model.eval()
with torch.no_grad():
    with tqdm(total=len(X_test_inputs_batches1), desc='Predicting', unit='batch') as pbar:
        for inputs_batch, attention_masks_batch in zip(X_test_inputs_batches1, X_test_attention_masks_batches1):
            inputs_batch = inputs_batch.to('cuda')
            attention_masks_batch = attention_masks_batch.to('cuda')

            y_pred_batch = scibert_model(inputs_batch, attention_mask=attention_masks_batch)[0].sigmoid().cpu().numpy()
            y_pred_list.append(y_pred_batch)

            pbar.update(1)

# Concatenate predictions from all batches
y_pred = np.concatenate(y_pred_list, axis=0)

# Convert predicted probabilities to binary predictions
y_pred_classes = (y_pred > 0.5).astype(int)

# Create a DataFrame with columns matching mlb.classes_
predictions_df = pd.DataFrame(y_pred_classes, columns=mlb.classes_)

# Concatenate the predictions DataFrame with the original test_df
final_df = pd.concat([test_df, predictions_df], axis=1)

# Display or save the final_df as needed
print(final_df)
final_df.to_csv('/kaggle/working/Brahmaputra.csv')


Predicting: 100%|██████████| 1220/1220 [03:12<00:00,  6.35batch/s]


          Id                                              Title  \
0      30332     Pricing FX Options under Intermediate Currency   
1      50337  A Multicore Processor based Real-Time System f...   
2      66515  Perceptual Quality Improvement in Videoconfere...   
3      57464  Hundred-Kilobyte Lookup Tables for Efficient S...   
4      43169  Efficient Sequence Labeling with Actor-Critic ...   
...      ...                                                ...   
10969  41708  A new lipid-structured model to investigate th...   
10970  38843  Evaluating the Efficacy of Hybrid Deep Learnin...   
10971  57571  Weakly Supervised Video Individual CountingWea...   
10972  31964  StructFormer: Learning Spatial Structure for L...   
10973  22229  Numerical Simulation guided Lazy Abstraction R...   

                                                Abstract  \
0      We suggest an intermediate currency approach t...   
1      In this paper we propose an Intelligent Manage...   
2      In the l